# FSRS4Anki v2.0.0 Optimizer

[![open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-spaced-repetition/fsrs4anki/blob/v2.0.0/fsrs4anki_optimizer.ipynb)

↑ Click the above button to open the optimizer on Google Colab.

> If you can't see the button and are located in the Chinese Mainland, please use a proxy or VPN.

Upload your **Anki Deck Package (.apkg)** file or **Anki Collection Package (.colpkg)** file on the `Left sidebar -> Files`, drag and drop your file in the current directory (not the `sample_data` directory). 

No need to include media. Need to include scheduling information. 

> If you use the latest version of Anki, please check the box `Support older Anki versions (slower/larger files)` when you export.

You can export it via `File -> Export...` or `Ctrl + E` in the main window of Anki.

Then replace the `filename` with yours in the next code cell. And set the `timezone` and `next_day_starts_at` which can be found in your preferences of Anki.

After that, just run all (`Runtime -> Run all` or `Ctrl + F9`) and wait for minutes. You can see the optimal parameters in section **3 Result**. Copy them, replace the parameters in `fsrs4anki_scheduler.js`, and paste them into the custom scheduling of your deck options (require Anki version >= 2.1.55).

**NOTE**: The default output is generated from my review logs. If you find the output is the same as mine, maybe your notebook hasn't run there.

In [1]:
# Here are some settings that you need to replace before running this optimizer.

filename = "ALL__Learning.apkg"
# If you upload deck file, replace it with your deck filename. E.g., ALL__Learning.apkg
# If you upload collection file, replace it with your colpgk filename. E.g., collection-2022-09-18@13-21-58.colpkg

# Replace it with your timezone. I'm in China, so I use Asia/Shanghai.
timezone = 'Asia/Shanghai'

# Replace it with your Anki's setting in Prefernces -> Scheduling.
next_day_starts_at = 4

# Replace it if you don't want the optimizer to use the review logs before a specific date.
revlog_start_date = "2006-10-05"


## 1 Build dataset

### 1.1 Extract Anki collection & deck file

In [2]:
import zipfile
# Extract the collection file or deck file to get the .anki21 database.
with zipfile.ZipFile(f'./{filename}', 'r') as zip_ref:
    zip_ref.extractall('./')
    print("Extract successfully!")


Extract successfully!


In [3]:
import sqlite3
import time
import tqdm
import pandas as pd
import os
from datetime import timedelta, datetime
from tqdm import tqdm


### 1.2 Create time-series feature

The following code cell will extract the review logs from your Anki collection and preprocess them to a trainset which is saved in `revlog_history.tsv`.

 The time-series features are important in optimizing the model's parameters. For more detail, please see my paper: https://www.maimemo.com/paper/

In [4]:
if os.path.isfile("collection.anki21b"):
    os.remove("collection.anki21b")
    raise Exception(
        "Please export the file with `support older Anki versions` if you use the latest version of Anki.")
elif os.path.isfile("collection.anki21"):
    con = sqlite3.connect("collection.anki21")
elif os.path.isfile("collection.anki2"):
    con = sqlite3.connect("collection.anki2")
else:
    raise Exception("Collection not exist!")
cur = con.cursor()
res = cur.execute("SELECT * FROM revlog")
revlog = res.fetchall()

df = pd.DataFrame(revlog)
df.columns = ['id', 'cid', 'usn', 'r', 'ivl',
              'last_lvl', 'factor', 'time', 'type']
df = df[(df['cid'] <= time.time() * 1000) &
        (df['id'] <= time.time() * 1000) &
        (df['id'] >= time.mktime(datetime.strptime(revlog_start_date, "%Y-%m-%d").timetuple()) * 1000)].copy()
df['create_date'] = pd.to_datetime(df['cid'] // 1000, unit='s')
df['create_date'] = df['create_date'].dt.tz_localize(
    'UTC').dt.tz_convert(timezone)
df['review_date'] = pd.to_datetime(df['id'] // 1000, unit='s')
df['review_date'] = df['review_date'].dt.tz_localize(
    'UTC').dt.tz_convert(timezone)
df.sort_values(by=['cid', 'id'], inplace=True, ignore_index=True)
df.to_csv("revlog.csv", index=False)
print("revlog.csv saved!")
df = df[(df['type'] == 0) | (df['type'] == 1)].copy()
df['real_date'] = df['review_date'].map(
    lambda x: x - timedelta(days=1) if x.hour < next_day_starts_at else x)
df['real_date'] = df['real_date'].dt.floor('D')
df.drop(df[df['real_date'].dt.year < 2006].index, inplace=True)
df.drop_duplicates(['cid', 'real_date'], keep='first', inplace=True)
df['delta_t'] = df.real_date.diff().dt.days
df.dropna(inplace=True)
df['delta_t'] = df['delta_t'].astype(dtype=int)
df['i'] = 1
df['r_history'] = ""
df['t_history'] = ""
col_idx = {key: i for i, key in enumerate(df.columns)}


# code from https://github.com/L-M-Sherlock/anki_revlog_analysis/blob/main/revlog_analysis.py
def get_feature(x):
    for idx, log in enumerate(x.itertuples()):
        if idx == 0:
            x.iloc[idx, col_idx['delta_t']] = 0
        if idx == x.shape[0] - 1:
            break
        x.iloc[idx + 1, col_idx['i']] = x.iloc[idx, col_idx['i']] + 1
        x.iloc[idx + 1, col_idx['t_history']] = f"{x.iloc[idx, col_idx['t_history']]},{x.iloc[idx, col_idx['delta_t']]}"
        x.iloc[idx + 1, col_idx['r_history']] = f"{x.iloc[idx, col_idx['r_history']]},{x.iloc[idx, col_idx['r']]}"
    return x


tqdm.pandas()
df = df.groupby('cid', as_index=False).progress_apply(get_feature)
df["t_history"] = df["t_history"].map(lambda x: x[1:] if len(x) > 1 else x)
df["r_history"] = df["r_history"].map(lambda x: x[1:] if len(x) > 1 else x)
df.to_csv('revlog_history.tsv', sep="\t", index=False)
print("Trainset saved!")


revlog.csv saved!


100%|██████████| 5166/5166 [00:17<00:00, 301.30it/s]


Trainset saved!


In [5]:
import math
import sys
import torch
import datetime
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from sklearn.utils import shuffle


The default parameters of FSRS.

In [6]:
initStability = 1
initStabilityRatingFactor = 1
initDifficulty = 1
initDifficultyRatingFactor = -1
updateDifficultyRatingFactor = -1
difficultyMeanReversionFactor = 0.2
recallFactor = 3
recallDifficultyDecay = -0.7
recallStabilityDecay = -0.2
recallRetrievabilityFactor = 1.2
forgetFactor = 2
forgetDifficultyDecay = -0.5
forgetStabilityDecay = 0.5
forgetRetrievabilityFactor = 1

## 2 Optimize parameter

### 2.1 Define the model

FSRS is a time-series model for predicting memory states.

In [7]:
class FSRS(nn.Module):
    def __init__(self):
        super(FSRS, self).__init__()
        self.f_s = nn.Parameter(torch.FloatTensor([initStability, initStabilityRatingFactor]))
        # init stability
        self.f_d = nn.Parameter(torch.FloatTensor([initDifficulty, initDifficultyRatingFactor, updateDifficultyRatingFactor, difficultyMeanReversionFactor]))
        # init difficulty
        self.s_w = nn.Parameter(torch.FloatTensor([recallFactor, recallDifficultyDecay, recallStabilityDecay, recallRetrievabilityFactor, forgetFactor, forgetDifficultyDecay, forgetStabilityDecay, forgetRetrievabilityFactor]))
        self.zero = torch.FloatTensor([0.0])

    def forward(self, x, s, d):
        '''
        :param x: [review interval, review response]
        :param s: stability
        :param d: difficulty
        :return:
        '''
        if torch.equal(s, torch.FloatTensor([0.0])):
            # first learn, init memory states
            next_s = self.f_s[0] * (self.f_s[1] * (x[1] - 1) + 1)
            next_d = self.f_d[0] * (self.f_d[1] * (x[1] - 4) + 1)
        else:
            r = torch.exp(np.log(0.9) * x[0] / s)
            if x[1] > 1:
                next_s = s * (1 + torch.exp(self.s_w[0]) * torch.pow(d, self.s_w[1]) *
                              torch.pow(s, self.s_w[2]) *
                              (torch.exp((1 - r) * self.s_w[3]) - 1))
            else:
                next_s = self.s_w[4] * torch.pow(d, self.s_w[5]) * torch.pow(s, self.s_w[6]) * (torch.exp((1 - r) * self.s_w[7]) - 1)
            next_d = d + self.f_d[2] * (x[1] - 3)
            next_d = self.mean_reversion(self.f_d[0] * (- self.f_d[1] + 1), next_d)
        return next_s, self.constrain(next_d)

    def loss(self, s, t, r):
        return - (r * np.log(0.9) * t / s + (1 - r) * torch.log(1 - torch.exp(np.log(0.9) * t / s)))

    def constrain(self, d):
        return torch.relu(d - 1) + 1

    def mean_reversion(self, init, current):
        return self.f_d[3] * init + (1-self.f_d[3]) * current


class WeightClipper(object):
    def __init__(self, frequency=1):
        self.frequency = frequency

    def __call__(self, module):
        if hasattr(module, 'f_s'):
            w = module.f_s.data
            w[0] = w[0].clamp(0.1, 10)  # initStability
            w[1] = w[1].clamp(0.01, 10)  # initStabilityRatingFactor
            module.f_s.data = w
        if hasattr(module, 'f_d'):
            w = module.f_d.data
            w[0] = w[0].clamp(1, 10)  # initDifficulty
            w[1] = w[1].clamp(-10, -0.01)  # initDifficultyRatingFactor
            w[2] = w[2].clamp(-10, -0.01)  # updateDifficultyRatingFactor
            w[3] = w[3].clamp(0, 1)  # difficultyMeanReversionFactor
            module.f_d.data = w
        if hasattr(module, 's_w'):
            w = module.s_w.data
            w[0] = w[0].clamp(0, 5)  # recallFactor
            w[1] = w[1].clamp(-2, -0.01)  # recallDifficultyDecay
            w[2] = w[2].clamp(-2, -0.01)  # recallStabilityDecay
            w[3] = w[3].clamp(0.01, 2)  # recallRetrievabilityFactor
            w[4] = w[4].clamp(0, 5)  # forgetFactor
            w[5] = w[5].clamp(-2, -0.01)  # forgetDifficultyDecay
            w[6] = w[6].clamp(0.01, 1)  # forgetStabilityDecay
            w[7] = w[7].clamp(0.01, 2)  # forgetRetrievabilityFactor
            module.s_w.data = w


def lineToTensor(line):
    ivl = line[0].split(',')
    response = line[1].split(',')
    tensor = torch.zeros(len(response), 2)
    for li, response in enumerate(response):
        tensor[li][0] = int(ivl[li])
        tensor[li][1] = int(response)
    return tensor


### 2.2 Train the model

The `revlog_history.tsv` generated before will be used for training the FSRS model.

In [8]:
model = FSRS()
clipper = WeightClipper()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

dataset = pd.read_csv("./revlog_history.tsv", sep='\t', index_col=None)
dataset = dataset[(dataset['i'] > 1) & (dataset['delta_t'] > 0) & (dataset['t_history'].str.count('0') == 1)]
dataset['tensor'] = dataset.progress_apply(lambda x: lineToTensor(
    list(zip([x['t_history']], [x['r_history']]))[0]), axis=1)
print("Tensorized!")

n_epoch = 1
print_len = max(dataset.shape[0] // 10, 1)

checkpoint = {
    "net": model.state_dict(),
    'optimizer': optimizer.state_dict(),
    "epoch": -1
}

for k in range(n_epoch):
    dataset = shuffle(dataset, random_state=2022 + k)
    epoch_len = len(dataset)
    for i, (_, row) in enumerate(tqdm(dataset.iterrows(), total=epoch_len, desc="train", colour="red")):
        model.train()
        optimizer.zero_grad()
        output_t = [(model.zero, model.zero)]
        for input_t in row['tensor']:
            output_t.append(model(input_t, *output_t[-1]))
        loss = model.loss(output_t[-1][0], row['delta_t'],
                          {1: 0, 2: 1, 3: 1, 4: 1}[row['r']])
        if np.isnan(loss.data.item()):
            # Exception Case
            print(row, output_t)
            raise Exception('error case')
        loss.backward()
        optimizer.step()
        model.apply(clipper)

        if (k * epoch_len + i) % print_len == 0:
            print(f"iteration: {k * epoch_len + i + 1}")
            for name, param in model.named_parameters():
                print(f"{name}: {list(map(lambda x: round(float(x), 4),param))}")

            checkpoint = {
                "net": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "iteration": (k * epoch_len + i) // print_len
            }

torch.save(checkpoint, f'./model.pth')

initStability, initStabilityRatingFactor = map(
    lambda x: round(float(x), 4), dict(model.named_parameters())['f_s'].data)
initDifficulty, initDifficultyRatingFactor, updateDifficultyRatingFactor, difficultyMeanReversionFactor = map(
    lambda x: round(float(x), 4), dict(model.named_parameters())['f_d'].data)
recallFactor, recallDifficultyDecay, recallStabilityDecay, recallRetrievabilityFactor, forgetFactor, forgetDifficultyDecay, forgetStabilityDecay, forgetRetrievabilityFactor = map(
    lambda x: round(float(x), 4), dict(model.named_parameters())['s_w'].data)

print("\nTraining finished!")


100%|██████████| 41465/41465 [00:02<00:00, 20246.99it/s]


Tensorized!


train:   0%|          | 47/41465 [00:00<01:28, 468.94it/s]

iteration: 1
f_s: [1.0005, 1.0]
f_d: [1.0, -0.9995, -1.0, 0.2005]
s_w: [3.0005, -0.6995, -0.1995, 1.2005, 2.0, -0.5, 0.5, 1.0]


train:  10%|█         | 4221/41465 [00:08<01:12, 511.72it/s]

...

iteration: 33169
f_s: [1.5876, 1.9115]
f_d: [1.0201, -0.8908, -1.0166, 0.0462]
s_w: [3.1043, -0.8095, -0.1732, 1.2921, 2.6186, -0.0166, 0.6375, 1.519]


train:  90%|█████████ | 37415/41465 [01:11<00:07, 537.66it/s]

iteration: 37315
f_s: [1.5562, 1.9238]
f_d: [1.0321, -0.9162, -1.0214, 0.0288]
s_w: [3.1003, -0.8299, -0.1865, 1.288, 2.6416, -0.0122, 0.6278, 1.5292]


train: 100%|██████████| 41465/41465 [01:19<00:00, 524.34it/s]

iteration: 41461
f_s: [1.5661, 1.9562]
f_d: [1.0073, -0.9025, -0.9889, 0.0268]
s_w: [3.1409, -0.8018, -0.1803, 1.3262, 2.6636, -0.0249, 0.622, 1.5335]

Training finished!


## 3 Result

Copy the optimal parameters for FSRS for you in the output of next code cell after running.

The scheduler code of FSRS4Anki is at https://github.com/open-spaced-repetition/fsrs4anki/blob/main/fsrs4anki_scheduler.js

In [9]:
print(f"let f_s = [{initStability},{initStabilityRatingFactor}];")
print(f"let f_d = [{initDifficulty},{initDifficultyRatingFactor},{updateDifficultyRatingFactor},{difficultyMeanReversionFactor}];")
print(f"let s_w = [{recallFactor},{recallDifficultyDecay},{recallStabilityDecay},{recallRetrievabilityFactor},{forgetFactor},{forgetDifficultyDecay},{forgetStabilityDecay},{forgetRetrievabilityFactor}];")

let f_s = [1.5663,1.9562];
let f_d = [1.0083,-0.9032,-0.9891,0.0262];
let s_w = [3.1401,-0.8024,-0.1813,1.3254,2.6638,-0.0247,0.6221,1.5337];


You can see the memory states and intervals generated by FSRS as if you press the good in each review at the due date scheduled by FSRS.

In [10]:
requestRetention = 0.9  # recommended setting: 0.8 ~ 0.9


class Collection:
    def __init__(self):
        self.model = model

    def states(self, t_history, r_history):
        with torch.no_grad():
            line_tensor = lineToTensor(list(zip([t_history], [r_history]))[0])
            output_t = [(self.model.zero, self.model.zero)]
            for input_t in line_tensor:
                output_t.append(self.model(input_t, *output_t[-1]))
            return output_t[-1]


my_collection = Collection()
print("1:again, 2:hard, 3:good, 4:easy\n")
for first_rating in (1,2,3,4):
    print(f'first rating: {first_rating}')
    t_history = "0"
    d_history = "0"
    r_history = f"{first_rating}"  # the first rating of the new card
    # print("stability, difficulty, lapses")
    for i in range(15):
        states = my_collection.states(t_history, r_history)
        # print('{0:9.2f} {1:11.2f} {2:7.0f}'.format(
            # *list(map(lambda x: round(float(x), 4), states))))
        next_t = round(float(np.log(requestRetention)/np.log(0.9) * states[0]))
        difficulty = round(float(states[1]), 1)
        t_history += f',{int(next_t)}'
        d_history += f',{difficulty}'
        r_history += f",3"
    print(f"rating history: {r_history}")
    print(f"interval history: {t_history}")
    print(f"difficulty history: {d_history}")
    print('')


1:again, 2:hard, 3:good, 4:easy

first rating: 1
rating history: 1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
interval history: 0,2,4,7,13,23,38,61,96,147,220,323,465,658,916,1258
difficulty history: 0,3.7,3.7,3.6,3.6,3.6,3.5,3.5,3.4,3.4,3.4,3.3,3.3,3.2,3.2,3.2

first rating: 2
rating history: 2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
interval history: 0,5,10,19,35,62,105,171,271,417,625,918,1321,1867,2596,3556
difficulty history: 0,2.8,2.8,2.8,2.8,2.7,2.7,2.7,2.7,2.7,2.6,2.6,2.6,2.6,2.6,2.5

first rating: 3
rating history: 3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
interval history: 0,8,18,39,78,147,262,447,734,1165,1793,2688,3934,5636,7921,10940
difficulty history: 0,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9

first rating: 4
rating history: 4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
interval history: 0,11,34,91,217,469,936,1744,3070,5147,8275,12830,19271,28151,40123,55950
difficulty history: 0,1.0,1.0,1.1,1.1,1.1,1.1,1.1,1.2,1.2,1.2,1.2,1.2,1.3,1.3,1.3



You can change the `test_rating_sequence` to see the scheduling intervals in different ratings.

In [11]:
test_rating_sequence = "3,3,3,3,1,3,3,3,3,3,3,3,3"
requestRetention = 0.9  # recommended setting: 0.8 ~ 0.9
easyBonus = 1.3
hardInterval = 1.2

t_history = "0"
d_history = "0"
for i in range(len(test_rating_sequence.split(','))):
    rating = test_rating_sequence[2*i]
    last_t = int(t_history.split(',')[-1])
    r_history = test_rating_sequence[:2*i+1]
    states = my_collection.states(t_history, r_history)
    print(states)
    next_t = max(1,round(float(np.log(requestRetention)/np.log(0.9) * states[0])))
    if rating == '4':
        next_t = round(next_t * easyBonus)
    elif rating == '2':
        next_t = round(last_t * hardInterval)
    t_history += f',{int(next_t)}'
    difficulty = round(float(np.log(requestRetention)/np.log(0.9) * states[1]), 1)
    d_history += f',{difficulty}'
print(f"rating history: {test_rating_sequence}")
print(f"interval history: {t_history}")
print(f"difficulty history: {d_history}")

(tensor(7.6943), tensor(1.9190))
(tensor(18.4257), tensor(1.9190))
(tensor(39.0190), tensor(1.9190))
(tensor(77.9732), tensor(1.9190))
(tensor(6.5330), tensor(3.8454))
(tensor(12.0728), tensor(3.7949))
(tensor(20.6513), tensor(3.7458))
(tensor(34.4183), tensor(3.6979))
(tensor(54.9393), tensor(3.6513))
(tensor(85.7533), tensor(3.6059))
(tensor(130.6474), tensor(3.5617))
(tensor(194.6362), tensor(3.5186))
(tensor(284.1125), tensor(3.4767))
rating history: 3,3,3,3,1,3,3,3,3,3,3,3,3
interval history: 0,8,18,39,78,7,12,21,34,55,86,131,195,284
difficulty history: 0,1.9,1.9,1.9,1.9,3.8,3.8,3.7,3.7,3.7,3.6,3.6,3.5,3.5
